# Funciones de orden superior

In [2]:
// Importamos librerias para testing
import $ivy.`org.scalatest::scalatest:3.0.8`
import _root_.org.scalatest._

import $ivy.$                               

import _root_.org.scalatest._

## `FoldRight`: estrategia para algoritmos de divide y vencerás

Consideremos varias funciones que aplican un patrón de divide y vencerás

In [1]:
def sum(list: List[Int]): Int = 
    list match {
        case Nil => 0
        case head :: tail => 
            val tailSol: Int = sum(tail)
            head + tailSol
    }

defined function sum

In [4]:
def multiply(list: List[Int]): Int = 
    list match {
        case Nil => 1
        case head :: tail => 
            val tailSol: Int = multiply(tail)
            head * tailSol
    }

defined function multiply

In [5]:
def someEven(list: List[String]): Boolean = 
    list match {
        case Nil => false
        case head :: tail =>
            val tailSol: Boolean = someEven(tail)
            head.length % 2 == 0 || tailSol
    }

defined function someEven

Ambas funciones son éxactamente iguales en cuanto a su estructura y en lo único que varian es en el elemento que es devuelto en caso de `Nil` y el operador que usamos en cada caso (`+` o `*`), así pues podemos abstraer esta estrcutura en un método mucho más general:

In [6]:
def combine(list: List[Int])(nil: Int, f: (Int, Int) => Int): Int = 
    list match {
        case Nil => nil
        case head :: tail => 
            val tailSol: Int = combine(tail)(nil, f)
            f(head, tailSol)
    }

defined function combine

Este método abstrae la estrucutra principal de nuestras funciones, de modo que sus argumentos son la estructura sobre la que operamos (una lista en nuestro caso), el elemento a devolver en caso de `Nil` y una función lambda que describa el comportamiento específico

Así podemos redefinir el comportamiento de las funciones `sum` y `multiply`

In [6]:
// Podemos definir la función lambda de forma normal
def sum(list: List[Int]): Int = 
    combine(list)(0, (a, b) => a + b)

defined function sum

In [7]:
// O podemos definirla abreviada
def multiply(list: List[Int]): Int = 
    combine(list)(1, _ * _)

defined function multiply

Podemos así generalizar nuestra función `combine` y obtener así la función `foldRight`

In [7]:
def foldRight[A, B](list: List[A])(nil: B, f: (A, B) => B): B = 
    list match {
        case Nil => nil
        case head :: tail => 
            val tailSol: B = foldRight(tail)(nil, f)
            f(head, tailSol)
    }

defined function foldRight

### Funcionamiento de `foldRight`

Gráficamente el comportamiento de `foldRight` se muestra como sigue:

Podemos así entender el funcionamiento de `foldRight` como una implementación del algoritmo de divide y vencerás, es decir, primero se divide el problema en subproblemas más pequeños y faciles de resolver, de modo que en segundo lugar, estos problemas son resueltos y finalmente, todos los problemas son combinados para obtener la solución final. En el caso de que los subproblemas no puedan ser resultos de forma directa, se resolverán de forma recursiva. En el caso de las listas podemos describir el siguiente comportamiento:\
- El problema es obtener un valor de tipo $B$ de una lista dada.
- El único subproblema le corresponde a la cola de la lista.
- Los argumentos de `foldRight` nos indican por tanto cómo obtener la solucion para una lista vacia (problema atómico), y cómo obtener la solución para el subproblema 

Así de este modo, podemos implementar las funciones `sum` y `multiply` mediante `foldRight`

In [9]:
def sum(list: List[Int]): Int = 
    foldRight[Int, Int](list)(
        0, // Solución al problema atómico
        (head, subsol) => head + subsol // composición para la resolución del subproblema
    )

defined function sum

In [10]:
def multiply(list: List[Int]): Int = 
    foldRight(list)(
        1, // Solución al problema atómico
        (head: Int, subsol: Int) => head * subsol // composición para la resolución del subproblema
    )

defined function multiply

### Mejora en la inferencia de tipos

Como podemo apreciar, necesitamos definir los tipos de nuestra función lambda para que Scala pueda compilar. Así pues una definición de este tipo no compilaria.

In [10]:
/*
def multiply(list: List[Int]): Int = 
    foldRight(list)(1, (a, b) => a * b)
*/

Para poder ayudar a Scala a inferir los tipos, podemos cambiar la signatura de `foldRight` currificandola por completo

In [8]:
def foldRight[A, B](list: List[A])(nil: B)(f: (A, B) => B): B = 
    list match {
        case Nil => nil
        case head :: tail => 
            val tailSol: B = foldRight(tail)(nil)(f)
            f(head, tailSol)
    }

defined function foldRight

Así de esta manera, la definición anterior consigue compilar

In [9]:
def multiply(list: List[Int]): Int = 
    foldRight(list)(1)((a, b) => a * b)

defined function multiply

### Funciones de orden superior en la API de Scala

Al contrario de como se ha programado anteriormente `foldRight`, recibiendo una lista como primer argumento, la API de Scala implementa esta función como un método dentro de la clase `List[A]`. El uso por tanto de este método es como sigue:

In [10]:
List(1,2,3).foldRight(0)(_ + _)

res9: Int = 6

## Funciones de orden superior en otros tipos de datos 

Podemos encontrar multitud de funciones de orden superior para listas

In [16]:
def foldLeft[A, B](list: List[A])(b: B)(f: (B, A) => B): B = ???
def filter[A](list: List[A])(f: A => Boolean): List[A] = ???
def map[A, B](list: List[A])(f: A => B): List[B] = ???
def flatMap[A, B](list: List[A])(f: A => List[B]): List[B] = ???

defined function foldLeft
defined function filter
defined function map
defined function flatMap

Pero también podemos encontrar similares para otros tipos de datos como `Option` o `Either`

In [19]:
// HOF -> Option[A]
def fold[A, B](opt: Option[A])(none: B)(some: A => B): B = ???
def filter[A](opt: Option[A])(f: A => Boolean): Option[A] = ???
def map[A, B](opt: Option[A])(f: A => B): Option[B] = ???  
def flatMap[A, B](opt: Option[A])(f: A => Option[B] ): Option[B] = ???

defined function fold
defined function filter
defined function map
defined function flatMap

In [18]:
// HOF -> Either[A,B]
def fold[A, B, C](opt: Either[A, B])(left: A => C, right: B => C): C = ???
def filter[A, B](opt: Either[A, B])(f: B => Boolean): Either[A, B] = ???
def map[A, B, C](opt: Either[A, B])(f: B => C): Either[A, C] = ???  
def flatMap[A, B, C](opt: Either[A, B])(f: B => Either[A, C] ): Either[A, C] = ???

defined function fold
defined function filter
defined function map
defined function flatMap

## `FoldLeft` para implementar algoritmos iterativos

Consideremos las siguientes funciones iterativas

In [13]:
def length[A](l: List[A]): Int = {
    var out: Int = 0
    for (e <- l)
        out += 1
    out
}

defined function length

In [14]:
length(List(1 , 2 , 3 , 4 , 5 , 6))

res13: Int = 6

In [15]:
def reverse[A](l: List[A]): List[A] = {
    var reverseList: List[A] = Nil
    for(e <- l)
        reverseList = e :: reverseList
    reverseList
}

defined function reverse

In [16]:
reverse(List(1 , 2 , 3 , 4 , 5 , 6))

res15: List[Int] = List(6, 5, 4, 3, 2, 1)

Como podemos apreciar, al igual que pasaba con los algoritmos de divide y vencerás, los algoritmos iterativos tienen una estructura común de modo que solo varía la variable mutable que será la solución, y el algoritmo que se realiza dentro del bucle. Así pues podemos definir un método `foldLeft` que abstraiga este comportamiento:

In [18]:
def foldLeft[A,B](l: List[A])(initial: B)(update: (B, A) => B): B = {
    var out: B = initial
    for (e: A <- l)
        update(out, e)
    out
}

defined function foldLeft

Podemos optimizar la implementación mediante recursión de cola:

In [21]:
def foldLeftTR[A,B](l: List[A])(initial: B)(update: (B, A) => B): B = {
    
    def foldLeftTRAux(l: List[A])(out: B)(update: (B, A) => B): B = {
        l match {
            case Nil => out
            case head :: tail => foldLeftTRAux(tail)(update(out, head))(update)
        }
    }
    
    foldLeftTRAux(l)(initial)(update)
}

defined function foldLeftTR

Así, podemos implementar ahora las funciones anteriores mediante `foldLeft`

In [19]:
def length[A](l: List[A]): Int = {
    foldLeft[A, Int](l)(0)((out: Int, _: A) => out + 1)
}

defined function length

In [22]:
def reverse[A](l: List[A]): List[A] = {
    foldLeft[A, List[A]](l)(Nil)((out: List[A], e: A) => e :: reverseList)
}

defined function reverse

Pero también podemos implementarlas mediante `foldRight`:

In [23]:
def lenght[A](l: List[A]): Int = {
    l.foldRight(0)((head: A, tailSol: Int) => tailSol + 1)
}

defined function lenght

In [24]:
def reverse[A](l: List[A]): List[A] = {
    l.foldRight(Nil: List[A])((head: A, tailSol: List[A]) => tailSol :+ head)
}

defined function reverse

## Función `map`

Esta función permite que a todos los elementos de una estructura de datos se les apliqué una modificación, de modo que no modificaremos la forma de la estructutra (si tenemos una lista, seguiremos teniendo una lista), si no el contenido. Así pues su comportamiento puede ser descrito de la siguiente manera:

Sea $map(s[A])(f(A))$ una función de $s[A] \rightarrow s[B]$ que recibe como argumentos, una estructura que contiene datos de tipo A ($s[A]$) y una función de modificación para cada uno de los elementos de $s[A]$, $f(A)$ tal que $A \rightarrow B$

Así se deducen las siguiente caracteristicas (en el caso de las listas, pero es extendible a cualquier estructura de datos):

- $map(list)(identity)=list$, $\forall list$ de tipo $Lista[A]$, siendo $A$ un elemento genérico y lista una estructura que contiene elemento de tipo $A$
- $map(map(list)(f))(g)=map(list)(g \circ f)$, $\forall list$ de tipo $Lista[A]$, $f: A \rightarrow B$, $g: B \rightarrow C$

Gráficamente se puede mostrar el comportamiento de `map` como sigue:

Su implementación para `list` en Scala es la siguiente:

In [ ]:
def map[A, B](list: List[A])(f: (A) => B): List[B] = {
    list match {
        case Nil => Nil
        case head :: tail => f(head) :: map(tail)(f)
    }
}

Así pues, podemos usar `map` en un ejemplo:

In [33]:
// Función que dada una lista de strings, cambien los que contienen una cadena par por '1'
// y los que contienen una cadena impar por '0
map[String, Int](List("Hola", "que", "tal", "yo", "bien", "y", "tu"))((s: String) => {
    if (s.length % 2 == 0) 1
    else 0
})

// También podemos usar los métodos propios dento de la API de Scala
List("Hola", "que", "tal", "yo", "bien", "y", "tu").map((s: String) => {
    if (s.length % 2 == 0) 1
    else 0
})

res32_0: List[Int] = List(1, 0, 0, 1, 1, 0, 1)
res32_1: List[Int] = List(1, 0, 0, 1, 1, 0, 1)

### Uso de `map` en `Option`

No solo podemos implementar `map` en `list`, si no también en muchas otras estructuras de datos, en el caso de `option` su implementación sería la siguiente:

In [26]:
def mapOpt[A,B](opt: Option[A])(f: (A) => B): Option[B] = {
    opt match {
        case None => None
        case Some(a: A) => Some(f(a): B)
    }
}

defined function mapOpt

## `filter` para filtrado de elementos

A diferencia de `map`, `filter` sí que permite modificar la forma de nuestra estructura de datos de modo que sean eliminados elementos que no cumplan una condición

Su comportamiento gráfico se puede mostrar tal que:

Su implementación por tanto en Scala sería:

In [27]:
def filter[A](list: List[A])(f: (A) => Boolean): List[A] = {
    list match {
        case Nil => Nil
        case head :: tail if f(head) => head :: filter(tail)(f)
        case _ :: tail => filter(tail)(f)
    }
}

defined function filter

Así podemos usar `filter` con un ejemplo:

In [35]:
// Dada una lista de números, buscamos eliminar todos aquellos que no son múltiplos de 3
filter[Int](List(0,11,54,27,56,18,146,12,78))((n: Int) => {
    n % 3 == 0
})

// Tambíen podemos usar los métodos de la API de Scala
List(0,11,54,27,56,18,146,12,78).filter(_ % 3 == 0)

res34_0: List[Int] = List(0, 54, 27, 18, 12, 78)
res34_1: List[Int] = List(0, 54, 27, 18, 12, 78)

### Uso de `filter` en `Option`

Al igual que habiamos explicado con `map`, `filter` también puede ser usado en multitud de estructuras de datos

Podemos implementarlo como sigue:

In [36]:
def filterOpt[A](opt: Option[A])(f: (A) => Boolean): Option[A] = {
    opt match {
        case None => None
        case Some(a: A) if f(a) => Some(a)
        case _ => None
    }
}

defined function filterOpt

## Uso de `flatMapping` en estructuras de datos